# Lexical search of Hyginus, *Fabulae*

> *Add link here to all the caveats...*


### How to use this notebook

1. First, run step 1 (e.g., by selecting the cell labelled **Step 1: load everything** and choosing "Run all below" from the "Cell" menu).  This will be slow, and your mileage may vary depending on how well your connection to different resources on the internet happens to be performing just then.
2. Just below the cell labelled **Step 2: search by example**, fill in between quotation marks an argument to the function `moreLikeThis`.  E.g., to find all forms of `duco` in Hyginus, you could edit that cell to read

    `moreLikeThis("duxit")`

Then run the cell (e.g., by selecting it, and choosing "Run cells" from the "Cell" menu).

# Step 2: search by example

In [ ]:
moreLikeThis("duci")

# Step 1: load everything


In [ ]:
// set up notebook to find repository
val personalRepo = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(personalRepo)

In [ ]:
// ivy imports
import $ivy.`edu.holycross.shot::latincorpus:7.0.0-pr5`

In [ ]:
import edu.holycross.shot.latincorpus._

In [ ]:
val hyginusUrl = "https://raw.githubusercontent.com/LinguaLatina/analysis/master/data/hyginus/hyginus-latc.cex"
val hyginus = LatinCorpus.fromUrl(hyginusUrl)

In [ ]:
val lewisShortBase = "http://folio2.furman.edu/lewis-short/index.html?urn=urn:cite2:hmt:ls.markdown:"
val hyginusPages = "https://lingualatina.github.io/texts/browsable/hyginus/"
val ambiguousCss = "text-decoration-line: underline; text-decoration-style: wavy; text-decoration-color: silver;"

def moreLikeThis(s: String) = {
    try {
        val lexemes = hyginus.tokenLexemeIndex(s)     
        val labelledLexemes = hyginus.tokenLabelledLexemeIndex(s)  
        val cns = hyginus.citableUnits.sequences.filter( cn => cn.matchesLexeme(lexemes))
        val heading = s"<h2>String <em>${s}</em> matched ${labelledLexemes.size} lexeme(s) in ${cns.size} passages</h2>" 
    

        val xlinks = for ( zipped <- lexemes.zipWithIndex) yield {
          val lex = zipped._1
          val idx = zipped._2
          val lsId = lex.replaceFirst("ls.", "")
          val lsUrl =lewisShortBase + lsId
          "<li><a href=\""  + lsUrl + "\">"  + labelledLexemes(idx) + "</a></li>"
        }
        val itemsString = xlinks.mkString("\n")
        
        val lexList = s"<ul>${itemsString}</ul>"
      
        val highlighter = LexemesHighlighter(lexemes, "color:rgb(24, 23, 162);") 
             
        val htmlBody = {
          val bullets = for ((cn) <- cns) yield {
            val link = cn match {
              case citable: LatinCitableUnit => {
              val psg = citable.canonicalNode.urn.collapsePassageBy(1)
              "<a style=\"color:rgb(159, 69, 17);\" href =\"" + hyginusPages + psg.passageComponent + "/\">Hyginus " + citable.canonicalNode.urn.passageComponent + "</a>"
              }
              case _ => ""
            }

            val ref = cn match {
              case citable: LatinCitableUnit => "<span style=\"color:rgb(159, 69, 17);\">" + link + "</span>"
              case _ => ""
            }

            val txtRaw = cn.lexicalText.mkString(" ")
            val txt = StringFormatter.tokensLexemeStyled(
              cn.tokens,
              highlighter,
              lexemes,
              formAmbiguityStyle = "", lexicalAmbiguityStyle = ambiguousCss, unanalyzedStyle = ""
            )
            s"<li>${ref} ${txt}</li>"
          }
          "<h3>Passages</h3><ol>" + bullets.mkString("\n") + "</ol>"
        }
        
      
      
        val key = "<h2>Key</h2><ul>" + 
        "<li style=\"color:blue;\">matched</li>" + 
        "<li style=\"text-decoration-line: underline; text-decoration-style: wavy;text-decoration-color: silver;\">multiple lexemes</li>" +
        "</ul>"
      

        Html(key + heading + lexList + htmlBody)
        //key + heading + lexList + htmlBody
        
        
    } catch {
       case nope: java.util.NoSuchElementException => {
       Html("<p><em>" + s + "</em> does not occur in Hyginus.  Please check the form, and try again.</p>")
       }    
    }

}
